In [43]:
import mpl_toolkits
import matplotlib.pyplot as plt
import pandas as pd
import geopandas as gpd
import numpy as np
import pickle
import os
import logging
from obspy import read
import time


from obspy.clients.fdsn.mass_downloader import RectangularDomain, Restrictions, MassDownloader
from scipy import signal
from obspy import UTCDateTime
from obspy.clients.fdsn import Client as FDSN_Client
from obspy import read_inventory
import asyncio
from itertools import islice
from itertools import tee

In [82]:
events_df = pd.read_pickle('data/events_processed.pkl')
stations_df = pd.read_pickle('data/stations_processed.pkl')

#events_full = events_df[(events_df.time > '2016-01-01') & (events_df.time < '2017-01-01')]
events_full = events_df
events_full.shape
folder = "normal" # normal/active

In [67]:
events = events_full[0:1000]
stations_df.shape

(60, 4)

In [62]:
mdl = MassDownloader(providers=['GEONET'])

[2022-05-10 18:02:20,766] - obspy.clients.fdsn.mass_downloader - INFO: Initializing FDSN client(s) for GEONET.
[2022-05-10 18:02:20,766] - obspy.clients.fdsn.mass_downloader - INFO: Initializing FDSN client(s) for GEONET.
[2022-05-10 18:02:20,766] - obspy.clients.fdsn.mass_downloader - INFO: Initializing FDSN client(s) for GEONET.
[2022-05-10 18:02:20,766] - obspy.clients.fdsn.mass_downloader - INFO: Initializing FDSN client(s) for GEONET.
[2022-05-10 18:02:20,770] - obspy.clients.fdsn.mass_downloader - INFO: Successfully initialized 1 client(s): GEONET.
[2022-05-10 18:02:20,770] - obspy.clients.fdsn.mass_downloader - INFO: Successfully initialized 1 client(s): GEONET.
[2022-05-10 18:02:20,770] - obspy.clients.fdsn.mass_downloader - INFO: Successfully initialized 1 client(s): GEONET.
[2022-05-10 18:02:20,770] - obspy.clients.fdsn.mass_downloader - INFO: Successfully initialized 1 client(s): GEONET.


In [63]:
def mass_data_downloader(start, stop, event_id, Station,
                         Network='NZ', 
                         Channel='HHZ', 
                         Location=10,
                         folder='normal'
                         ):
    '''
    This function uses the FDSN mass data downloader to automatically download
    data from the XH network deployed on the RIS from Nov 2014 - Nov 2016.
    More information on the Obspy mass downloader available at:
    https://docs.obspy.org/packages/autogen/obspy.clients.fdsn.mass_downloader.html
    Inputs:
    start: "YYYYMMDD"
    stop:  "YYYYMMDD"
    Network: 2-character FDSN network code
    Station: 2-character station code
    Channel: 3-character channel code
    Location: 10.
    '''
    #print("=" * 65)
    #print("Initiating mass download request.")
    #print('downloading: ',event_id)

    domain = RectangularDomain(
        minlatitude=-47.749,
        maxlatitude=-33.779,
        minlongitude=166.104,
        maxlongitude=178.990
    )

    restrictions = Restrictions(
        starttime = start,
        endtime = stop,
        chunklength_in_sec = None,
        network = Network,
        station = Station,
        location = Location,
        channel = Channel,
        reject_channels_with_gaps = False,
        minimum_length = 0.0,
        minimum_interstation_distance_in_m = 100.0
    )

    #mdl = MassDownloader(providers=['GEONET'])
    ev_str = str(event_id).replace(":", "_")
    mdl.download(
        domain,
        restrictions,
        mseed_storage=f"datasets/{folder}/waveforms/{ev_str}",
        stationxml_storage=f"datasets/{folder}/stations",
    )
    #print('done: ',event_id)

logger = logging.getLogger("obspy.clients.fdsn.mass_downloader")
logger.setLevel(logging.WARNING)

In [64]:
#async def final_download():
#    for i, event in events.iterrows():
#        event_id = event.event_id
#        event_time = event['time']  
#        start=event_time - T_event
#        end=event_time + H_event
#        
#        print("=" * 65)
#        print("Initiating mass download request.")
#        print(event_id)
#        #asyncio.run(main())
#        #tasks = [download_st(start, end, event_id, station) for j, station in stations_df.iterrows()]
#        
#        #tasks = [asyncio.to_thread(mass_data_downloader, start, end, event_id, station.station_code) for j, station in stations_df.iterrows()]
#        #print(len(tasks))
#        #await asyncio.gather(*tasks)
#        stations = ",".join([station.station_code for j, station in stations_df.iterrows()])
#        mass_data_downloader(start, end, event_id, stations)
#

In [83]:
async def final_download_threaded(events):
    time1 = time.perf_counter()
    tasks = []
    print("Initiating mass download request.")
    for i, event in events.iterrows():
        event_id = event.event_id
        event_time = event['time']
        start=event_time - T_event - H_event
        end=event_time - H_event
        #stations = ",".join([station.station_code for j, station in stations_df.iterrows()])
        stations = event['closest_station']
        tasks.append(asyncio.to_thread(mass_data_downloader, start, end, event_id, stations, folder=folder))
        
    await asyncio.gather(*tasks)
    time2 = time.perf_counter()
    print(f"{threads_at_once} threads took {time2-time1:0.1f} seconds")
    
#VARIABLES FOR DOWNLOAD

T_event = 30
if folder == "normal": H_event = 1000
else: H_event = 10 #10
threads_at_once = 100
print(folder, H_event)
for event_sublist in [events[x:x+threads_at_once] for x in range(0, len(events), threads_at_once)]:
    await final_download_threaded(event_sublist)
print("Done!")

normal 1000
Initiating mass download request.
100 threads took 1.8 seconds
Done!


In [32]:
for i in pd.interval_range(start=100, end=501, freq=30, closed='right').to_tuples():
    print(i)

(100, 130)
(130, 160)
(160, 190)
(190, 220)
(220, 250)
(250, 280)
(280, 310)
(310, 340)
(340, 370)
(370, 400)
(400, 430)
(430, 460)
(460, 490)


In [38]:
def pairwise(iterable):
    "s -> (s0,s1), (s1,s2), (s2, s3), ..."
    a, b = tee(iterable)
    next(b, None)
    return zip(a, b)
min([abs(x-y) for x, y in pairwise(events['time'])])

210.958631